<a href="https://colab.research.google.com/github/lvllvl/SLAM/blob/main/unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Section

## Run script: Connect to google drive

In [13]:
# Mount your drive, to save parts of your model, e.g., checkpoints, etc.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Run Script: Git clone repos

- comma10k repo
- project repo (SLAM)

### Run requirements.txt

## clone repo, org folder structures

In [15]:
import os
import shutil

def clone_repo(repo_url, dest_path):
    if os.path.exists(dest_path):
        print(f"Directory {dest_path} already exists. Removing and re-cloning.")
        shutil.rmtree(dest_path)
    os.system(f"git clone {repo_url} {dest_path}")

# Clone repos
clone_repo('https://github.com/lvllvl/SLAM.git', 'SLAM')
clone_repo('https://github.com/commaai/comma10k.git', 'comma10k')


# import os
# import shutil

def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

def move_files(src, dst):
    if not os.path.exists(src):
        print(f"Source directory {src} does not exist.")
        return
    for filename in os.listdir(src):
        src_file = os.path.join(src, filename)
        dst_file = os.path.join(dst, filename)
        try:
            shutil.move(src_file, dst_file)
        except Exception as e:
            print(f"Error moving {src_file} to {dst_file}: {e}")


def move_selected_files(src, dst, prefix=None, suffix=None):
    """
    Move files from src to dst based on prefix and/or suffix criteria.

    :param src: Source directory
    :param dst: Destination directory
    :param prefix: Prefix to filter files (optional)
    :param suffix: Suffix to filter files (optional)
    """
    if not os.path.exists(src):
        print(f"Source directory {src} does not exist.")
        return
    for filename in os.listdir(src):
        if prefix and not filename.startswith(prefix):
            continue
        if suffix and not filename.endswith(suffix):
            continue
        src_file = os.path.join(src, filename)
        dst_file = os.path.join(dst, filename)
        try:
            shutil.move(src_file, dst_file)
        except Exception as e:
            print(f"Error moving {src_file} to {dst_file}: {e}")


# Create directories for training and validation
create_directory('dataset_root/train/images')
create_directory('dataset_root/train/masks')
create_directory('dataset_root/val/images')
create_directory('dataset_root/val/masks')

# Move images and masks to training folder
move_files('comma10k/imgs', 'dataset_root/train/images')
move_files('comma10k/masks', 'dataset_root/train/masks')

# Move images and masks to validation folder
move_files('comma10k/imgs2', 'dataset_root/val/images')
move_files('comma10k/masks2', 'dataset_root/val/masks')

# Move SLAM *.py files to the dataset_root folder
move_selected_files( 'SLAM/', 'dataset_root/', prefix=None, suffix='.py' )

# List directories to verify
print(os.listdir('.'))
print(os.listdir('dataset_root'))


Directory SLAM already exists. Removing and re-cloning.
Directory comma10k already exists. Removing and re-cloning.


In [20]:
# Verify all same number of files are in the folders

!ls -l dataset_root/train/images | grep ^- | wc -l
!ls -l dataset_root/train/masks | grep ^- | wc -l

!ls -l dataset_root/val/images | grep ^- | wc -l
!ls -l dataset_root/val/masks | grep ^- | wc -l

9888
9888
2000
2000


# Model Section

## Segmentation Model: U-net Segmentaiton

In [21]:
!python dataset_root/train.py

Traceback (most recent call last):
  File "/content/dataset_root/train.py", line 101, in <module>
    main()
  File "/content/dataset_root/train.py", line 82, in main
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
  File "/content/dataset_root/train.py", line 29, in train_one_epoch
    for images, masks in dataloader:
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 630, in __next__
    data = self._next_data()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 674, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_ind

In [ ]:
#!ls dataset_root/train/masks/3521_2665ce6f7b1dce08_2018-10-05--18-18-06_6_82_mask.png
!ls dataset_root/train/masks/3521*

dataset_root/train/masks/3521_2665ce6f7b1dce08_2018-10-05--18-18-06_6_82.png
